In [21]:
import tensorflow as tf

# ignore warnings   
import warnings
warnings.filterwarnings('ignore')


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
directory_data_train = "data/data4/train"
directory_data_test = "data/data4/validation"

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

train_ds = train_datagen.flow_from_directory(
    directory=directory_data_train,
    batch_size=32,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    subset="training",
    seed=123  
)

validation_ds = val_datagen.flow_from_directory(
    directory=directory_data_train,
    batch_size=32,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    subset="validation",
    seed=123 
)



Found 963 images belonging to 14 classes.
Found 232 images belonging to 14 classes.


In [23]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Tải mô hình MobileNetV2 đã được huấn luyện trước mà không bao gồm lớp phân loại đầu ra
MobileNetV2_base = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg')  # 'pooling='avg' sẽ tự động thêm lớp GlobalAveragePooling2D

# Đóng băng các lớp của mô hình đã được huấn luyện trước
MobileNetV2_base.trainable = False

# Xây dựng mô hình sử dụng Functional API
x = MobileNetV2_base.output

x = BatchNormalization()(x)
x = Dropout(0.3)(x)  # Giảm Dropout để giữ lại nhiều đặc trưng hơn
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)


# Lớp đầu ra
predictions = Dense(train_ds.num_classes, activation='softmax')(x)

# Định nghĩa mô hình
model = tf.keras.Model(inputs=MobileNetV2_base.input, outputs=predictions)

# Biên dịch mô hình
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [24]:

history= model.fit(train_ds,
    validation_data=validation_ds,
    steps_per_epoch=len(train_ds),
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='accuracy',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 30s 789ms/step - accuracy: 0.3372 - loss: 2.2262 - val_accuracy: 0.8405 - val_loss: 0.8565
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 145ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8405 - val_loss: 0.8565
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 737ms/step - accuracy: 0.8481 - loss: 0.5402 - val_accuracy: 0.8922 - val_loss: 0.4559
Epoch 4/50


2024-08-15 01:58:09.521397: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 151ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8922 - val_loss: 0.4559
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 741ms/step - accuracy: 0.8968 - loss: 0.3602 - val_accuracy: 0.9310 - val_loss: 0.2917
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 147ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9310 - val_loss: 0.2917
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 750ms/step - accuracy: 0.9255 - loss: 0.2242 - val_accuracy: 0.9267 - val_loss: 0.2493
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 145ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9267 - val_loss: 0.2493
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 751ms/step - accuracy: 0.9343 - loss: 0.2265 - val_accuracy: 0.9181 - val_loss: 0.2514
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 146ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9181 - val_loss: 0.2514
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 744ms/step - accuracy: 0.9573 - loss

2024-08-15 02:06:40.827426: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9397 - val_loss: 0.2078
Epoch 37/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 738ms/step - accuracy: 0.9712 - loss: 0.0862 - val_accuracy: 0.9483 - val_loss: 0.1672
Epoch 38/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9483 - val_loss: 0.1672
Epoch 39/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 30s 780ms/step - accuracy: 0.9749 - loss: 0.0742 - val_accuracy: 0.9353 - val_loss: 0.2361
Epoch 40/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 162ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9353 - val_loss: 0.2361
Epoch 41/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 28s 786ms/step - accuracy: 0.9709 - loss: 0.1214 - val_accuracy: 0.9526 - val_loss: 0.2056
Epoch 42/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 160ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9526 - val_loss: 0.2056
Epoch 43/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 769ms/step - accuracy: 0.9709 -

In [25]:
# model.summary()

In [26]:
import pickle
# Lưu lịch sử huấn luyện
with open('history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [27]:
# Lưu mô hình
model.save("model.h5")
print(tf.__version__)

2.17.0
